## What is the meaning of churn prediction?
It the technique using which we can predict or detect the customers who  are likely to cancel their subscription in the near future. 

There are 3 ways using which businesses can generate revenue:
1. Upsell to existing customers.
2. Acquire new customers. 
3. Increase a customer retention.

Churn prediction helps in focusing in the third point i.e. increase a customer retention in which we can predict the churn rate or those customers who are going to cancel the subscription and then take pro-active action before they leave so that they stay in business.

### why we are doing this project?
In this project we will be creating a model using the bank past churn data of the cutomers and predict the future churn rate of customers so that we can know which cateogry of customer has has high churn rate and take a pro-active action inorder to make them stay in business.

### What is churn rate?
Accoring to wikipedia, The churn rate is the rate at which customers stop doing business with an entity. It is most commonly expressed as the percentage of service subscribers who discontinue their subscriptions within a given time period.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as po
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/bank-customer-churn-modeling/Churn_Modelling.csv")
df.drop(['RowNumber','CustomerId'], axis=1, inplace=True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

## EDA

#### Exited

In [ ]:
df['Exited'].value_counts()

In [ ]:
sns.countplot(df['Exited'])

In [ ]:
df['Exited'].value_counts().keys().tolist(), df['Exited'].value_counts().values

In [ ]:
# visulaize the Exited data using plotly
plot_by_exited_labels = df['Exited'].value_counts().keys().tolist()
plot_by_exited_values = df['Exited'].value_counts().values.tolist()

data = [
    go.Pie(labels=plot_by_exited_labels,
          values=plot_by_exited_values,
          hole=.6)
]

plot_layout = go.Layout(dict(title="Customer Churn"))

fig = go.Figure(data=data, layout=plot_layout)
po.iplot(fig)

we can see that our data is imbalanced.

In [ ]:
df.groupby("Exited").mean()

We can also see from the above means of different feature that is they are having significant impact in the churn data.

### Categorical features

In [ ]:
categorical_feat = ['Geography','Tenure','NumOfProducts',"HasCrCard","HasCrCard","IsActiveMember"]
for cat in categorical_feat:
    print(cat," : ")
    print(df[cat].value_counts())
    sns.countplot(df[cat])
    plt.show() 

### Gender

In [ ]:
print(df['Gender'].value_counts()/df.shape[0])
sns.countplot(df['Gender'])
plt.show() 

out of total, almost 55% of them are male and 45% are female

In [ ]:
# Gender vs Exited
# total percentage exited in by each gender
plot_by_gender = df.groupby('Gender')['Exited'].mean().reset_index()
print(plot_by_gender)

plot_data = [
    go.Bar(
    x = plot_by_gender['Gender'],
    y = plot_by_gender['Exited'])
]
plot_layout = go.Layout(dict(title="% of Exited customers in each gender"))
fig = go.Figure(data = plot_data, layout=plot_layout)
po.iplot(fig)

### Geography

In [ ]:
print(df['Geography'].value_counts())
sns.countplot(df['Geography'])
plt.show() 

out of 3 countries provided, France has highest churn rate 

In [ ]:
# Geography vs Exited in percentage
# total percentage of Exited customer percentage in each Geography
plot_by_geo = df.groupby('Geography')['Exited'].mean().reset_index()
print(plot_by_geo)

plot_data = [
    go.Bar(
    x = plot_by_geo['Geography'],
    y = plot_by_geo["Exited"])
]
plot_layout = go.Layout(dict(title="Percentage of CUstomer Exited in each Geography"))
fig = go.Figure(data=plot_data, layout=plot_layout)
po.iplot(fig)

### NumOfProducts

In [ ]:
print(df['NumOfProducts'].value_counts())
sns.countplot(df['NumOfProducts'])

We can see most of the customers has used only one product, and very less customer has used 4 of the bank product.

In [ ]:
df['NumOfProducts'].value_counts()/df.shape[0]*100

we can see that less then 1 % of customer has used all 4 products.

In [ ]:
sns.countplot(df['NumOfProducts'],hue=df['Exited'])

In [ ]:
pd.crosstab(index=df['Exited'], columns=df['NumOfProducts'])

In [ ]:
# lets draw % of Exited Customer along with NumOfProducts used
plot_by_numOfProducts = df.groupby("NumOfProducts")["Exited"].mean().reset_index()
print(plot_by_numOfProducts)

plot_data = [
    go.Bar(
    x = plot_by_numOfProducts["NumOfProducts"],
    y = plot_by_numOfProducts["Exited"])
]
plot_layout = go.Layout(dict(title = "% Exited customers with NumOfProducts used"))
fig = go.Figure(data=plot_data,
               layout=plot_layout)
po.iplot(fig)

we can conclude that all the customers that have used all the 4 products have left which is 1 i.e. 100%.

### HasCrCard

In [ ]:
print(df['HasCrCard'].value_counts())
sns.countplot(df['HasCrCard'], hue=df['Exited'])

from above, customers with no credit card are more on leaving the business.

In [ ]:
# lets plot % of Exited customer with HasCrCard
plot_with_hasCrCard = df.groupby("HasCrCard")["Exited"].mean().reset_index()
print(plot_with_hasCrCard)

plot_data = [
    go.Bar(
    x=plot_with_hasCrCard['HasCrCard'],
    y= plot_with_hasCrCard['Exited'])
]
plot_layout = go.Layout(dict(title="% Exited customer with HasCrCard"))
fig = go.Figure(data=plot_data, layout=plot_layout)
po.iplot(fig)

We don't see much of the difference between the Exited customer between both category as both have almost equal number of exited customer so we won't get  much of the impact from this feature to our model.

### Balance

In [ ]:
# Balance
for i in [0,1]:
    sns.distplot(df[df['Exited']==i]['Balance'])
    plt.show()

As we can see both the exited and non exited customer are having show how same distribution of data.

### Correlation

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(df.corr(), annot=True)

### Scaling

In [ ]:
df.drop('Surname', axis=1, inplace=True)
df.head()

In [ ]:
df = pd.get_dummies(data=df, columns=['Geography','Gender','HasCrCard','IsActiveMember'])
df.head()

In [ ]:
x = df.drop('Exited', axis=1)
y = df['Exited']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x[['CreditScore','Age','Tenure','Balance','EstimatedSalary','NumOfProducts']] = scaler.fit_transform(x[['CreditScore','Age','Tenure','Balance','EstimatedSalary','NumOfProducts']])

In [ ]:
x.head()

### splitting

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

x_train, x_test,y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.shape, x_test.shape, Counter(y_train), Counter(y_test)

As we can see our data is imbalanced.

## Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve

In [ ]:
def generated_report(y_actual, y_pred):
    print("Accuracy : ", accuracy_score(y_actual, y_pred))
    print(classification_report(y_actual, y_pred))
    
def generated_roc_auc_curve(model, x_test):
    y_pred_proba = model.predict_proba(x_test)[:, 1]
    fpr, tpr, thresh = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test,  y_pred_proba)
    plt.plot(fpr, tpr, label='AUC: '+str(auc))
    plt.legend()
    plt.show()
    
def Log_Reg_Model():
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    generated_report(y_test, y_pred)
    generated_roc_auc_curve(lr, x_test)

## Without using sampling methods

In [ ]:
Log_Reg_Model()

Above is the scores without doing any of the sampling methods. We cannot just rely on accuracy score as it can be misleading sometimes so we need to look for the values of other socres like f1-score, precision, etc.

## RandomUnderSampling

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [ ]:
sampler = RandomUnderSampler(sampling_strategy=1, replacement=False)
x_new, y_new = sampler.fit_resample(x, y)

Counter(y_new), x_new.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.2, stratify=y_new)
Counter(y_train), Counter(y_test)

In [ ]:
Log_Reg_Model()

we can see its score is overall better then the previous one  without balancing our data but in case of undersampling there will be alot of data still not used for making the model that causes info loss and our model cannot generalize the result well in case of future prediction

## RandomOverSampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

x_new, y_new = RandomOverSampler(sampling_strategy=0.8).fit_resample(x, y)
Counter(y_new)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.2, stratify=y_new)

In [ ]:
Log_Reg_Model()

OUr accuracy has little decreased but some score have definitely increased cause we are not just looking for accuracy score.

## SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

smote = SMOTE(sampling_strategy=0.87)
x_new, y_new = smote.fit_resample(x, y)

print(Counter(y))
print(Counter(y_new))

x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.2, stratify=y_new)
Counter(y_train), Counter(y_test)

In [ ]:
Log_Reg_Model()

We can see that our score has increased, but using SMOTE can be misleading as we are generating systhetic data.

## UnderSampling and OverSampling

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
from sklearn.linear_model import LogisticRegression


cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
pipeline = Pipeline(steps=[
    ("over",SMOTE(sampling_strategy=0.6)),
    ("under", RandomUnderSampler(sampling_strategy=0.9))
])

In [ ]:
print("Before: ", Counter(y))

x_new, y_new = pipeline.fit_resample(x, y)
print("After: ",Counter(y_new))

In [ ]:
x_new.shape, y_new.shape

In [ ]:
# again
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
pipeline = Pipeline(steps=[
    ("over",SMOTE(sampling_strategy=0.6)),
    ("under", RandomUnderSampler(sampling_strategy=0.9)), 
    ("model", LogisticRegression())
])

#evaluate pipeline
scores = cross_val_score(pipeline, x, y, scoring='roc_auc', cv=cv)
print(scores)
print("mean roc auc: ", np.mean(scores))

In [ ]:
pipeline = Pipeline(steps=[
    ("over",SMOTE(sampling_strategy=0.6)),
    ("under", RandomUnderSampler(sampling_strategy=0.9))
])

x_new, y_new = pipeline.fit_resample(x, y)
x_new.shape, y_new.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.2, stratify=y_new)
Counter(y_train), Counter(y_test)

In [ ]:
Log_Reg_Model()

As we can see using the over and under sampling combined has given pretty good result of accuracy then all other sampling technique that we have used before.

The significane of using this combined method is that using over sampling will help the generation of synthetic data to the less cateogry data and under sampling will under sample the category data that is more in number according to the sampling_strategy paramter speficied due to which helps to balance the data from being bias to one category due to lack of data and also add some more synthetic data to the sample when may have biasness if we have used RandomOverSampler.

### Thank you
#### The is my first notebook in kaggle.
### Upvote is appreciated